In [110]:
import numpy as np
import pickle
import random
from sklearn.metrics import pairwise_distances
from sklearn.utils import shuffle
from sklearn.linear_model import Ridge
import warnings
warnings.filterwarnings('ignore')

work_dir = '/Users/ronghao/Mirror/Cornell-Tech/2018-Fa-Course/CS-5785/Homework/Final/'
data_dir = work_dir + 'all/'

In [31]:
num_train = 8000; num_dev = 2000; num_test = 2000

In [6]:
# Models['ridge']['X']['Y'][rand_state] = (train_score, dev_score, model)
Models = pickle.load(open(work_dir + "modeldata/Models_train_dev_score.p", "rb"))

In [79]:
KNN_Models = pickle.load(open(work_dir + "modeldata/KNN_dev_score_model.p", "rb"))

In [34]:
X_dict = pickle.load(open(work_dir + "modeldata/X_dict.p", "rb"))
Y_dict = pickle.load(open(work_dir + "modeldata/Y_dict.p", "rb"))

In [19]:
dist_discount_dev_dict = pickle.load(open(work_dir + "modeldata/dist_discount_dev_dict.p", "rb"))
dist_discount_test = pickle.load(open(work_dir + "modeldata/dist_discount_test.p", "rb"))

In [7]:
X_data = ['only_fc', 'only_pool']#, 'fc_pool'
Y_data = ['full_w2v_mean', 'n_full_w2v_mean', 'a_full_w2v_mean', 'v_full_w2v_mean',\
                          'n_top800_w2v_mean', 'a_top400_w2v_mean', 'v_top400_w2v_mean',\
                          'n_top400_w2v_mean', 'a_top200_w2v_mean', 'v_top200_w2v_mean',\
         'full_bow', 'n_full_bow', 'a_full_bow', 'v_full_bow',\
                     'n_top800_bow', 'a_top400_bow', 'v_top400_bow',\
                     'n_top400_bow', 'a_top200_bow', 'v_top200_bow']

In [12]:
# performance_by_train_score = []
# for X_datakey in X_data:    
#     for Y_datakey in Y_data:
#         train_score = Models['ridge'][X_datakey][Y_datakey][0][0]
#         dev_score = np.mean([Models['ridge'][X_datakey][Y_datakey][i][1] for i in range(4)])
#         model = Models['ridge'][X_datakey][Y_datakey][0][2]
#         performance_by_train_score.append((train_score, dev_score, X_datakey, Y_datakey, model))
# performance_by_train_score = sorted(performance_by_train_score, reverse=True)

# performance_by_dev_score = []
# for X_datakey in X_data:    
#     for Y_datakey in Y_data:
#         train_score = Models['ridge'][X_datakey][Y_datakey][0][0]
#         dev_score = np.mean([Models['ridge'][X_datakey][Y_datakey][i][1] for i in range(4)])
#         model = Models['ridge'][X_datakey][Y_datakey][0][2]
#         performance_by_dev_score.append((dev_score, train_score, X_datakey, Y_datakey, model))
# performance_by_dev_score = sorted(performance_by_dev_score, reverse=True)

# Ridge models
performance_by_dev_score = []
for X_datakey in X_data:    
    for Y_datakey in Y_data:
        train_score = Models['ridge'][X_datakey][Y_datakey][0][0]
        dev_score = np.mean([Models['ridge'][X_datakey][Y_datakey][i][1] for i in range(4)])
        model = Models['ridge'][X_datakey][Y_datakey][0][2]
        performance_by_dev_score.append((dev_score, X_datakey, Y_datakey, model))
performance_by_dev_score = sorted(performance_by_dev_score, reverse=True)

In [78]:
X_effective_data = ['only_fc', 'only_pool']#, 'fc_pool'
Y_effective_data = ['full_w2v_mean', 'n_full_w2v_mean', 'a_full_w2v_mean', 'v_full_w2v_mean',\
                     'full_bow', 'n_full_bow', 'a_full_bow', 'v_full_bow']

In [80]:
# KNN models
knn_by_dev_score = []
for X_datakey in X_effective_data:    
    for Y_datakey in Y_effective_data:
        dev_score = KNN_Models[X_datakey][Y_datakey][8][0]
        knn_by_dev_score.append((dev_score, X_datakey, Y_datakey))
knn_by_dev_score = sorted(knn_by_dev_score, reverse=True)

In [142]:
for i in knn_by_dev_score:
    print(i[0]/2, i[1], i[2])

0.31789083859681766 only_pool full_w2v_mean
0.3160623626158843 only_pool n_full_bow
0.31089521510803436 only_fc n_full_bow
0.3101287697177693 only_pool n_full_w2v_mean
0.30958673027071826 only_fc full_w2v_mean
0.3053516074321705 only_pool full_bow
0.3010714493259578 only_fc n_full_w2v_mean
0.29465637998912 only_fc full_bow
0.151210206062857 only_pool v_full_bow
0.15055249527519265 only_fc v_full_bow
0.13962289670146674 only_pool a_full_bow
0.13737618369332766 only_pool v_full_w2v_mean
0.13665205614603837 only_fc v_full_w2v_mean
0.12952267274886275 only_fc a_full_bow
0.10696290761719361 only_pool a_full_w2v_mean
0.10560497086118217 only_fc a_full_w2v_mean


### Load full data model

In [81]:
Ridge_all_data_Models = pickle.load(open(work_dir + "modeldata/Ridge_all_data_model.p", "rb"))
KNN_all_data_Models = pickle.load(open(work_dir + "modeldata/KNN_all_data_Models.p", "rb"))

In [93]:
Ridge_dev_score_full_model = []
for t in performance_by_dev_score:
    if t[3] in Y_effective_data:
        Ridge_dev_score_full_model.append((t[0], t[2], t[3], Ridge_all_data_Models[t[2]][t[3]]))

In [92]:
KNN_dev_score_full_model = []
for t in knn_by_dev_score:
    KNN_dev_score_full_model.append((t[0], t[1], t[2], KNN_all_data_Models[t[1]][t[2]]))

### Ensemble

In [43]:
model_list = [t for t in performance_by_dev_score[:5]]

In [61]:
model_list = performance_by_dev_score

In [60]:
[i[0] for i in performance_by_dev_score]

[0.3907993400970375,
 0.38903846914233425,
 0.38707619157212325,
 0.38480804832422477,
 0.3813549004497824,
 0.3795985334094225,
 0.37709139168741046,
 0.37696643816817,
 0.3765344810246513,
 0.37293766316618965,
 0.3720922219169075,
 0.37117333721865936,
 0.36779821729483103,
 0.36255521148796593,
 0.35856672361597375,
 0.35230960827742946,
 0.24246428876045267,
 0.24219797290826217,
 0.23790681025630717,
 0.23736450038770568,
 0.23239580213925223,
 0.23213608145649073,
 0.2070653651082105,
 0.20477380510141735,
 0.1975388718050096,
 0.19623172827344654,
 0.1952215479405785,
 0.195192727247705,
 0.19504650943580412,
 0.19323517761102416,
 0.192630013030163,
 0.1907576535849127,
 0.1905179114879144,
 0.1904466008234971,
 0.18970639488381844,
 0.18950187926538545,
 0.18919067022051733,
 0.18585011640137897,
 0.18262256765422397,
 0.1807253007806607]

In [65]:
ensemble_results_on_state = {}

for rand_state in range(1):
    print('test on state:', rand_state)
    ensemble_distance = np.zeros((2000,2000))
    for model_keys in model_list:
        x_train_dev = X_dict['train_dev'][model_keys[-3]]
        y_train_dev = Y_dict['train_dev'][model_keys[-2]]
        split_idx = list(range(num_train + num_dev))
        split_idx = shuffle(split_idx, random_state=rand_state)
        x_dev = x_train_dev[split_idx[num_train:]]
        y_dev = y_train_dev[split_idx[num_train:]]
        
        y_dev_pred = model_keys[-1].predict(x_dev)
        dev_distances_origin = pairwise_distances(y_dev, y_dev_pred)
        ensemble_distance = ensemble_distance + dev_distances_origin * model_keys[0] * dist_discount_dev_dict[rand_state]
    ensemble_distance = ensemble_distance
        
    dev_scores = []
    dev_pos_list = []

    for i in range(num_dev):
        pred_dist_idx = list(np.argsort(ensemble_distance[i]))
        dev_pos = pred_dist_idx.index(i)
        dev_pos_list.append(dev_pos)
        if dev_pos < 20:
            dev_scores.append(1 / (dev_pos + 1))
        else:
            dev_scores.append(0.0)
    ensemble_results_on_state[rand_state] = np.mean(dev_scores)

test on state: 0


In [66]:
ensemble_results_on_state

{0: 0.40570115230057185}

### Collect all the test distance matrix with dev score

In [100]:
test_distance_model_score = []

In [101]:
for model_keys in KNN_dev_score_full_model:
    print(model_keys[0], 'KNN', model_keys[1], model_keys[2])
    x_test = X_dict['test'][model_keys[1]]
    y_test = Y_dict['test'][model_keys[2]]
    y_test_pred = model_keys[-1].predict(x_test)
    test_distances_origin = pairwise_distances(y_test, y_test_pred)
    test_distance_model_score.append((model_keys[0], 'KNN', model_keys[1], model_keys[2], test_distances_origin))

0.6357816771936353 KNN only_pool full_w2v_mean
0.6321247252317685 KNN only_pool n_full_bow
0.6217904302160687 KNN only_fc n_full_bow
0.6202575394355386 KNN only_pool n_full_w2v_mean
0.6191734605414365 KNN only_fc full_w2v_mean
0.610703214864341 KNN only_pool full_bow
0.6021428986519156 KNN only_fc n_full_w2v_mean
0.58931275997824 KNN only_fc full_bow
0.302420412125714 KNN only_pool v_full_bow
0.3011049905503853 KNN only_fc v_full_bow
0.2792457934029335 KNN only_pool a_full_bow
0.2747523673866553 KNN only_pool v_full_w2v_mean
0.27330411229207674 KNN only_fc v_full_w2v_mean
0.2590453454977255 KNN only_fc a_full_bow
0.21392581523438722 KNN only_pool a_full_w2v_mean
0.21120994172236435 KNN only_fc a_full_w2v_mean


In [102]:
pickle.dump(test_distance_model_score, open(work_dir + "modeldata/test_distance_model_score.p", "wb"))

In [103]:
for model_keys in Ridge_dev_score_full_model:
    print(model_keys[0], 'Ridge', model_keys[1], model_keys[2])
    x_test = X_dict['test'][model_keys[1]]
    y_test = Y_dict['test'][model_keys[2]]
    y_test_pred = model_keys[-1].predict(x_test)
    test_distances_origin = pairwise_distances(y_test, y_test_pred)
    test_distance_model_score.append((model_keys[0], 'Ridge', model_keys[1], model_keys[2], test_distances_origin))

0.3907993400970375 Ridge only_pool n_full_bow
0.38707619157212325 Ridge only_pool full_bow
0.38480804832422477 Ridge only_pool full_w2v_mean
0.3813549004497824 Ridge only_pool n_full_w2v_mean
0.37709139168741046 Ridge only_fc n_full_bow
0.37696643816817 Ridge only_fc full_bow
0.37293766316618965 Ridge only_fc full_w2v_mean
0.3720922219169075 Ridge only_fc n_full_w2v_mean
0.24246428876045267 Ridge only_pool a_full_bow
0.24219797290826217 Ridge only_fc a_full_bow
0.2070653651082105 Ridge only_fc a_full_w2v_mean
0.20477380510141735 Ridge only_pool a_full_w2v_mean
0.1975388718050096 Ridge only_pool v_full_bow
0.19504650943580412 Ridge only_pool v_full_w2v_mean
0.19323517761102416 Ridge only_fc v_full_w2v_mean
0.192630013030163 Ridge only_fc v_full_bow


In [106]:
# for i in range(len(test_distance_model_score)):
#     if test_distance_model_score[i][1] == 'KNN':
#         test_distance_model_score[i] = (test_distance_model_score[i][0]/2, test_distance_model_score[i][1],\
#                                        test_distance_model_score[i][2], test_distance_model_score[i][3],\
#                                        test_distance_model_score[i][4])

In [108]:
pickle.dump(test_distance_model_score, open(work_dir + "modeldata/test_distance_model_score.p", "wb"))

### Combination Output on test

In [127]:
for t in range(19):
    n = random.randint(5, 10)
    idx = list(range(len(test_distance_model_score)))
    idx = shuffle(idx)
    print('-'.join([str(i) for i in idx[:n]]))

20-31-2-28-26
20-27-8-12-25-4
4-29-2-9-5
0-11-27-6-14-29-19
12-18-22-25-15-23-19-16-11
19-27-15-13-21-30-2-6-26-1
26-7-8-10-11-18
2-18-21-5-17-22-16-6-19
27-19-25-22-30-16-4-11
20-27-5-2-10-21-3
2-23-26-13-22-6-25-15-14
12-0-28-5-9-27-20-3-4
12-4-18-3-21
9-31-27-25-21-26-6-5-24-19
7-27-19-30-21-6-23-2-29-12
6-0-31-15-17-22
30-16-28-5-1-26
3-16-18-5-23
30-23-20-11-21-3


In [128]:
for t in range(19):
    n = random.randint(5, 10)
    idx = list(range(len(test_distance_model_score)))
    idx = shuffle(idx)
    ensemble_test_distance = np.zeros((2000,2000))
    for i in idx[:n]:
        ensemble_test_distance = ensemble_test_distance +\
                            test_distance_model_score[i][-1]*test_distance_model_score[i][0]
    ensemble_test_distance = ensemble_test_distance * dist_discount_test
    pred_rows = []

    for i in range(num_test):
        test_dist_idx = list(np.argsort(ensemble_test_distance[i]))
        top_20 = test_dist_idx[:20]
        row = ["%d.jpg" % i for i in test_dist_idx[:20]]
        pred_rows.append(" ".join(row))

    with open("random/ensemble_"+'-'.join([str(i) for i in idx[:n]])+"_submission.csv", "w") as f:
        f.write("Descritpion_ID,Top_20_Image_IDs\n")
        for i, row in enumerate(pred_rows):
            f.write("%d.txt,%s\n" % (i, row))

    print("Output written!")

Output written!
Output written!
Output written!
Output written!
Output written!
Output written!
Output written!
Output written!
Output written!
Output written!
Output written!
Output written!
Output written!
Output written!
Output written!
Output written!
Output written!
Output written!
Output written!


In [131]:
model_list = test_distance_model_score

In [138]:
ensemble_test_distance = np.zeros((2000,2000))
for model_keys in model_list:
    ensemble_test_distance = ensemble_test_distance + model_keys[-1]*(model_keys[0])**2.5
ensemble_test_distance = ensemble_test_distance * dist_discount_test**0.8
pred_rows = []

for i in range(num_test):
    test_dist_idx = list(np.argsort(ensemble_test_distance[i]))
    top_20 = test_dist_idx[:20]
    row = ["%d.jpg" % i for i in test_dist_idx[:20]]
    pred_rows.append(" ".join(row))

with open("ensemble_KNN_Ridge_ALL_2508_submission.csv", "w") as f:
    f.write("Descritpion_ID,Top_20_Image_IDs\n")
    for i, row in enumerate(pred_rows):
        f.write("%d.txt,%s\n" % (i, row))

print("Output written!")

Output written!


### Naive Output on test

In [94]:
model_list = KNN_dev_score_full_model[:5]

In [96]:
ensemble_test_distance = np.zeros((2000,2000))
for model_keys in model_list:
    print('a')
    x_test = X_dict['test'][model_keys[-3]]
    y_test = Y_dict['test'][model_keys[-2]]
    y_test_pred = model_keys[-1].predict(x_test)
    test_distances_origin = pairwise_distances(y_test, y_test_pred)
    ensemble_test_distance = ensemble_test_distance + test_distances_origin * model_keys[0]
ensemble_test_distance = ensemble_test_distance * dist_discount_test

pred_rows = []

for i in range(num_test):
    test_dist_idx = list(np.argsort(ensemble_test_distance[i]))
    top_20 = test_dist_idx[:20]
    row = ["%d.jpg" % i for i in test_dist_idx[:20]]
    pred_rows.append(" ".join(row))

with open("ensemble_discount_top_8NN_submission.csv", "w") as f:
    f.write("Descritpion_ID,Top_20_Image_IDs\n")
    for i, row in enumerate(pred_rows):
        f.write("%d.txt,%s\n" % (i, row))

print("Output written!")

a
a
a
a
a
Output written!


In [ ]:
top_5_8NN_dist = ensemble_test_distance